In [1]:
import os
from typing import Literal
from operator import itemgetter
from typing_extensions import TypedDict
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

True

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

### Third-party data

source data -> text loader -> embedding vector -> store vectors -> retrieve vectors

In [3]:
# # TODO: add third-party data
# loader = TextLoader("./data/recipes.json")
# index = VectorstoreIndexCreator(embedding=OpenAIEmbeddings()).from_loaders([loader]) 

# data = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# all_splits = text_splitter.split_documents(data)
# vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

# # k is the number of chunks to retrieve
# k = 4
# retriever = vectorstore.as_retriever(k=k)

In [4]:
# query = "How to make Classic Margherita Pizza?"
# result = index.query_with_sources(query, llm=OpenAI())

## Build chains for 4 use cases

In [5]:
# model
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [6]:
def chain_creator(llm, prompt_template):
    return prompt_template | llm | StrOutputParser()

In [7]:
# Define prompt templates for each service
tourist_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert in finding tourist attractions."),
        ("human", "Find top tourist attractions in {location}."),
    ]
)

itinerary_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert in travel itinerary planning."),
        ("human", "Create a 3-day itinerary for a visit to {location}."),
    ]
)

restaurant_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert in recommending restaurants."),
        ("human", "Suggest restaurants in {location} that offer {cuisine}."),
    ]
)

travel_ideas_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert in recommending travel ideas."),
        ("human", "Recommend travel destinations for someone interested in {interests}."),
    ]
)

In [8]:
# Create chains for each service
tourist_chain = chain_creator(llm, tourist_prompt)
itinerary_chain = chain_creator(llm, itinerary_prompt)
restaurant_chain = chain_creator(llm, restaurant_prompt)
travel_ideas_chain = chain_creator(llm, travel_ideas_prompt)

## Route an input query to a specific chain 

In [9]:
# Define the routing system
class RouteQuery(TypedDict):
    """Route query to the appropriate destination."""
    destination: Literal[
        "Tourist Attraction", 
        "Itinerary Planning", 
        "Restaurant Recommendations", 
        "Exploring Travel Ideas"
    ]

route_system = "Route the user's query to either 'Tourist Attraction', 'Itinerary Planning', 'Restaurant Recommendations', or 'Exploring Travel Ideas'."

route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", route_system),
        ("human", "{query}"),
    ]
)

# Routing chain based on the query
route_chain = route_prompt | llm.with_structured_output(RouteQuery) | itemgetter("destination")

# TODO: ParsedQuery output type based on usage
class ParsedQuery(TypedDict):
    """Extracted information from a natural language query."""
    location: str
    cuisine: str

# Step 1: Create a parsing chain to extract structured data (e.g., location, cuisine) from the natural language query
parse_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert travel assistant. Extract relevant information like location and cuisine from the query."),
        ("human", "{query}"),
    ]
)

# Chain to extract structured information from the natural query
parse_chain = parse_prompt | llm.with_structured_output(ParsedQuery)

In [10]:
# TODO: simplify this paragraph
# Final chain that routes to the appropriate LLM chain based on user input
def get_chain_for_service(service_type, query):
    if service_type == "Tourist Attraction":
        # Expecting 'location' variable
        return tourist_chain.invoke({"location": query.get("location")})
    elif service_type == "Itinerary Planning":
        # Expecting 'location' variable
        return itinerary_chain.invoke({"location": query.get("location")})
    elif service_type == "Restaurant Recommendations":
        # Expecting 'location' and 'cuisine' variables
        return restaurant_chain.invoke({"location": query.get("location"), "cuisine": query.get("cuisine")})
    elif service_type == "Exploring Travel Ideas":
        # Expecting 'interests' variable
        return travel_ideas_chain.invoke({"interests": query.get("interests")})
    else:
        raise ValueError(f"Unknown service type: {service_type}")

In [11]:
# Combine all the steps into the final chain
final_chain = {
    "destination": route_chain,
    "query": parse_chain
} | RunnableLambda(
    # Final step: Route to the correct service and invoke the right chain
    lambda x: get_chain_for_service(x["destination"], x['query'])
)

In [12]:
# Example query usage
if __name__ == "__main__":
    # Example of a natural language query
    inout_query = {
        "query": "Please list the top 3 Italian restaurants in New York"
    }
    
    # Pass the natural language query to the final chain
    response = final_chain.invoke(inout_query)
    print(f"Response: {response}")

Response: Here are some popular Italian restaurants in New York City:

1. Carbone - Located in Greenwich Village, Carbone offers classic Italian-American dishes in a stylish, retro setting.
2. L'Artusi - This West Village spot serves up modern Italian cuisine in a chic, bustling atmosphere.
3. Via Carota - A cozy Italian trattoria in the West Village known for its rustic dishes and welcoming ambiance.
4. Lupa - Mario Batali's Roman-inspired trattoria in Greenwich Village is a favorite for pasta lovers.
5. Marea - For upscale Italian seafood, check out Marea in Midtown Manhattan for a refined dining experience.
6. Il Buco - A Noho institution known for its seasonal, Mediterranean-inspired Italian fare and charming atmosphere.
7. Don Angie - This West Village gem offers a modern take on Italian-American cuisine with a creative twist.
8. Rubirosa - Located in Nolita, Rubirosa is a cozy spot known for its thin-crust pizza and classic Italian dishes.
9. Eataly - For a one-stop shop for all 